Antes de empezar 

[robots.txt](https://www.pisos.com/robots.txt)  
[html curso](https://www.w3schools.com/html/)

# Beautiful Soup Tutorial

Como científico de datos, tarde o temprano llegarás a un punto en el que tendrás que recopilar grandes cantidades de datos. Ya sea un proyecto o por pasatiempo y no siempre podremos contar con las API, pero tranquilo tenemos el web scraping... ¡Y una de las mejores herramientas de web scraping es Beautiful Soup!

## ¿Pero.... qué es el web scraping?
El web scraping consiste en obtener datos de páginas web de manera automatizada. Cuando visitas un sitio web, el servidor devuelve un documento HTML.   

En pocas palabras, el web scraping es la recopilación automatizada de datos de sitios web (para ser más precisos, del contenido HTML de los sitios web).

En este Jupyter, aprenderás los conceptos básicos sobre cómo extraer datos de HTML. 

El web scraping consiste en obtener datos de páginas web de manera automatizada. Cuando visitas un sitio web, el servidor devuelve un documento HTML. 
  

  

> **Nota ética:** Antes de raspar un sitio web, revisa sus términos de uso y su archivo `robots.txt` para asegurarte de que permites el scraping. No envíes muchas solicitudes en poco tiempo para no sobrecargar el servidor. Usa esta técnica de forma responsable.

### Conoce a tus nuevos mejores amigos: 

- Beautiful Soup
- Requests

In [ ]:
# Instala la librería beautifulsoup4 que permite parsear y navegar por documentos HTML/XML
# Esta es la herramienta principal para hacer web scraping
#%pip install beautifulsoup4

Para obtener la experiencia completa de Beautiful Soup, también deberás instalar un parser, dentro de ellos tenemos..

- html.parser
- lxml
- html5lib


Vamos a utilizar el lxml ya que es el mas rápido 

! + código → Ejecuta un comando del sistema (en una terminal) y lo ejecuta directamente.  
Se ejecuta en la primera versión de Python que el sistema encuentre, así que no tienes control sobre en qué entorno o versión se instala algo si tienes varios Python instalados.  

% + código → Ejecuta un comando mágico de Jupyter (o del kernel de VS Code) dentro del entorno o kernel activo, por lo que usa el mismo Python que está ejecutando tu notebook o script.  

In [ ]:
# Instala lxml, un parser HTML/XML muy rápido que BeautifulSoup utilizará
# para procesar el código HTML de manera eficiente
#!pip install lxml

Se necesita una cosa más para que podamos comenzar a hacer web scraping, y es la biblioteca de ```requests```. Con ```requests``` podemos solicitar páginas web de sitios web.

In [ ]:
# Instala la librería requests para hacer peticiones HTTP a sitios web
# Permite descargar el contenido HTML de las páginas web que queremos scrapear
#!pip install requests

Ahora asi manos a la obra..

## Mi primer scraping

Como siempre lo primero es importar las librerías 

In [1]:
# Importa las librerías necesarias para el web scraping y manipulación de datos
from bs4 import BeautifulSoup as bs  # Para parsear HTML
import requests  # Para hacer peticiones HTTP
import pandas as pd  # Para manipulación de datos (aunque no se usa en este notebook)
import numpy as np  # Para operaciones numéricas (aunque no se usa en este notebook)

Ahora, estamos listos para solicitar nuestra primera página web. No es nada complicado: guardamos la URL que queremos raspar en la variable URL, luego solicitamos la URL (requests.get (url)) y guardamos la respuesta en la variable de respuesta:

[web](https://www.weather-forecast.com/)

In [2]:
# Solicita al usuario que introduzca el nombre de una ciudad
# y construye la URL de weather-forecast.com para esa ciudad
city=str(input("Introduzca la ciudad:"))
url = "https://www.weather-forecast.com/locations/"+city+"/forecasts/latest"

In [3]:
# Muestra la URL construida para verificar que es correcta
url

'https://www.weather-forecast.com/locations/valencia/forecasts/latest'

In [4]:
# Realiza una petición HTTP GET a la URL especificada
# y almacena la respuesta del servidor en la variable response
response=requests.get(url)

In [5]:
# Verifica el código de estado HTTP de la respuesta
# 200 significa que la conexión fue exitosa y se obtuvo el contenido
response.status_code # ¡conexión correcta!

200

Posibles respuestas:

- [Respuestas informativas](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status#information_responses) (100–199)
- [Respuestas exitosas](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status#successful_responses) (200–299)
- [Mensajes de redirección](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status#redirection_messages) (300–399)
- [Respuestas de error del cliente](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status#client_error_responses) (400–499)
- [Respuestas de error del servidor](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status#server_error_responses) (500–599)

Pero necesitamos el contenido HTML de la página web solicitada, así que como siguiente paso guardamos el contenido de la respuesta a html:

In [6]:
# Extrae el contenido HTML de la respuesta en formato bytes
# Este es el código fuente de la página web que vamos a analizar
html = response.content

In [7]:
# Comprueba el tipo de dato de la variable html
# Muestra que es de tipo 'bytes' (datos binarios)
type(html)

bytes

Lo podemos imprimir para ver su estructura

In [8]:
# Imprime el HTML en bruto
# Se verá difícil de leer porque está sin formato y con muchas etiquetas mezcladas
print(html)

b'<!DOCTYPE html><html lang="en"><head><link crossorigin="" href="https://www.googletagmanager.com/" rel="preconnect" /><link href="https://www.googletagmanager.com/" rel="dns-prefetch" /><meta charset="utf-8" /><meta content="width=device-width" name="viewport" /><title>Valencia, Spain Weather Forecast</title><meta name="description" content="12 day Valencia, Spain Weather Forecast. Live Weather Warnings, hourly weather updates. Accurate Valencia weather today, forecast for sun, rain, wind and temperature." /><meta name="keywords" content="Valencia,Spain weather forecast,12 day weather forecast,Spain weather,weather today,hourly weather,Spain climate,weather warning,rain,snow,wind,humidity,UV index,temperature,www.weather-forecast.com" /><meta content="460425463983282" property="fb:app_id" /><link rel="stylesheet" media="all" href="https://www.weather-forecast.com/assets/application-a82b6935c112359c4a18a49dbd42a9b13cfe22f15108cd884b58f91427ccaed1.css" /><link rel="stylesheet" media="s

Este es el resultado obtenido en HTML de la página de la previsión metereológica, pero es realmente difícil de leer...

Pero para eso usamos BeautifulSoup y lxml

Cómo lo hacemos?..

Creamos un objeto BeautifulSoup llamado soup(sopa) con la siguiente línea de código:

In [9]:
# Crea un objeto BeautifulSoup que parsea el HTML usando lxml
# lxml es el parser elegido porque es el más rápido
# soup es el objeto que nos permitirá navegar y buscar en el HTML fácilmente
soup = bs(html, "lxml") # lxlm es un parser(corta en trozitos)

[documentacion](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

bs?

> from bs4 import BeautifulSoup as bs

El primer parámetro del método bs() es html (que fue la variable en la que guardamos ese contenido HTML difícil de leer).

El segundo parámetro ('lxml'), es el parser que se usa en html 

Ahora vamos a ver el cambio

In [10]:
# Imprime el HTML parseado por BeautifulSoup
# Ahora se ve más organizado y estructurado que el HTML en bruto
print(soup)

<!DOCTYPE html>
<html lang="en"><head><link crossorigin="" href="https://www.googletagmanager.com/" rel="preconnect"/><link href="https://www.googletagmanager.com/" rel="dns-prefetch"/><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><title>Valencia, Spain Weather Forecast</title><meta content="12 day Valencia, Spain Weather Forecast. Live Weather Warnings, hourly weather updates. Accurate Valencia weather today, forecast for sun, rain, wind and temperature." name="description"/><meta content="Valencia,Spain weather forecast,12 day weather forecast,Spain weather,weather today,hourly weather,Spain climate,weather warning,rain,snow,wind,humidity,UV index,temperature,www.weather-forecast.com" name="keywords"/><meta content="460425463983282" property="fb:app_id"/><link href="https://www.weather-forecast.com/assets/application-a82b6935c112359c4a18a49dbd42a9b13cfe22f15108cd884b58f91427ccaed1.css" media="all" rel="stylesheet"/><link href="https://www.weather-forecast

## Cómo navegar por un objeto de Beautiful Soup

HTML consta de elementos como enlaces, párrafos, encabezados, bloques, etc. Estos elementos están envueltos entre etiquetas; dentro de la etiqueta de apertura y cierre se puede encontrar el contenido del elemento.

Los elementos HTML también pueden tener atributos que contienen información adicional sobre el elemento. Los atributos se definen en las etiquetas de apertura con la siguiente sintaxis: nombre del atributo = "valor del atributo".

Ahora que hemos aprendido algo de HTML básico, finalmente podemos comenzar a extraer datos de soup. Simplemente escriba un nombre de etiqueta después de soup y un punto (como soup.title), y observe cómo se desarrolla la magia:

In [11]:
# Accede a la primera etiqueta <title> del HTML
# Devuelve el elemento completo con sus etiquetas de apertura y cierre
soup.title

<title>Valencia, Spain Weather Forecast</title>

In [12]:
# Accede a la primera etiqueta <h1> del HTML
# Devuelve el encabezado principal de la página con todos sus atributos
soup.h1

<h1 class="main-title__header"><span class="show-for-medium-up">Valencia (Valencia) Weather Forecast</span><span class="show-for-small-only">Valencia (Valencia) Weather Forecast</span></h1>

Eliminamos las etiquetas

In [13]:
# Extrae solo el texto contenido en la etiqueta <h1>, sin las etiquetas HTML
# get_text() elimina todas las etiquetas y devuelve solo el contenido textual
soup.h1.get_text()

'Valencia (Valencia) Weather ForecastValencia (Valencia) Weather\xa0Forecast'

Buscamos los datos que nos interesan: en este caso, etiquetas ```td``` y ```p```.

In [14]:
# Crea una lista vacía para almacenar las descripciones del tiempo
w_desc=[]

# Busca todas las etiquetas <td> con la clase especificada que contienen las descripciones del clima
for weather in soup.find_all('td', class_='b-forecast__table-description-cell--js'):
    # Dentro de cada <td>, busca la etiqueta <p> con la clase especificada y extrae su texto
    desc = weather.find('p', class_='b-forecast__table-description-content').text
    # Añade la descripción a la lista
    w_desc.append(desc)

In [15]:
# Muestra la lista con todas las descripciones del tiempo recopiladas
# Contiene predicciones meteorológicas de varios días
w_desc

['Mostly dry. Warm (max 30°C on Thu afternoon, min 19°C on Tue morning). Winds decreasing (strong winds from the W on Thu morning, light winds from the WSW by Thu night).',
 'Moderate rain (total 11mm), heaviest on Sun night. Warm (max 28°C on Sat afternoon, min 14°C on Sun night). Winds decreasing (fresh winds from the W on Sat afternoon, calm by Sun morning).',
 'Some drizzle, heaviest during Mon morning. Warm (max 22°C on Wed night, min 14°C on Mon morning). Wind will be generally light.',
 'Mostly dry. Warm (max 24°C on Thu afternoon, min 18°C on Sat morning). Wind will be generally light.']

Buscamos ahora datos dentro de la etiqueta ```tbody``` cuya ```class```es la indicada en el ejercicio.

In [16]:
# Crea una lista vacía para almacenar filas de datos de estaciones meteorológicas
alist=[]
# Busca la primera tabla con clase específica y recorre cada fila <tr>
for i in soup.find_all('tbody', class_="b-metar-table__body")[0]: # Los vientos
    print(i) # Imprime cada fila (elemento <tr>) que contiene etiquetas <td>
    alist.append((i).encode('utf-8'))  # Codifica cada fila en UTF-8 y la añade a la lista

<tr class="b-metar-table__row"><td class="b-metar-table__weather-station"><div><strong class="b-metar-table__weather-station-name"><a href="/weather-stations/El-Calvari-weather-station-1">El Calvari wx</a></strong> </div><div class="b-metar-table__weather-station-detail"><span class="dist">1</span> <span class="distu">km</span> <span class="dir">NE</span></div><div class="b-metar-table__weather-station-elevation"><span><span class="height">22</span><span class="heightu">m</span> alt.</span></div><div class="b-metar-table__weather-station-time">1 hour ago</div></td><td class="b-metar-table__temperature"><div class="b-metar-table__temperature-value temp-color4"><span class="temp">22.0</span>° <span class="tempu">C</span></div></td><td class="b-metar-table__weather"><div class="b-metar-table__weather-container"><div class="b-metar-table__weather-icon"></div><span class="b-metar-table__weather-text">Dry</span></div></td><td class="b-metar-table__wind"><div class="b-metar-table__wind-contai

In [17]:
# Muestra el primer elemento de la lista alist
# Contiene el HTML codificado en bytes de la primera fila de la tabla meteorológica
alist[0]

b'<tr class="b-metar-table__row"><td class="b-metar-table__weather-station"><div><strong class="b-metar-table__weather-station-name"><a href="/weather-stations/El-Calvari-weather-station-1">El Calvari wx</a></strong> </div><div class="b-metar-table__weather-station-detail"><span class="dist">1</span> <span class="distu">km</span> <span class="dir">NE</span></div><div class="b-metar-table__weather-station-elevation"><span><span class="height">22</span><span class="heightu">m</span> alt.</span></div><div class="b-metar-table__weather-station-time">1 hour ago</div></td><td class="b-metar-table__temperature"><div class="b-metar-table__temperature-value temp-color4"><span class="temp">22.0</span>\xc2\xb0\xe2\x80\x89<span class="tempu">C</span></div></td><td class="b-metar-table__weather"><div class="b-metar-table__weather-container"><div class="b-metar-table__weather-icon"></div><span class="b-metar-table__weather-text">Dry</span></div></td><td class="b-metar-table__wind"><div class="b-meta

In [18]:
# Guarda la primera fila (primer elemento) de la tabla en una variable
city_weather_info=alist[0] # vamos al primer tr
# Crea un nuevo objeto BeautifulSoup parseando solo esa fila específica
soup1 = bs(city_weather_info,'lxml')

In [19]:
# Muestra el objeto soup1 parseado
# Contiene solo la información de la primera estación meteorológica en formato HTML legible
soup1

<html><body><tr class="b-metar-table__row"><td class="b-metar-table__weather-station"><div><strong class="b-metar-table__weather-station-name"><a href="/weather-stations/El-Calvari-weather-station-1">El Calvari wx</a></strong> </div><div class="b-metar-table__weather-station-detail"><span class="dist">1</span> <span class="distu">km</span> <span class="dir">NE</span></div><div class="b-metar-table__weather-station-elevation"><span><span class="height">22</span><span class="heightu">m</span> alt.</span></div><div class="b-metar-table__weather-station-time">1 hour ago</div></td><td class="b-metar-table__temperature"><div class="b-metar-table__temperature-value temp-color4"><span class="temp">22.0</span>° <span class="tempu">C</span></div></td><td class="b-metar-table__weather"><div class="b-metar-table__weather-container"><div class="b-metar-table__weather-icon"></div><span class="b-metar-table__weather-text">Dry</span></div></td><td class="b-metar-table__wind"><div class="b-metar-table_

Obtenemos los datos contenidos en la fila

In [20]:
# Extrae y muestra información específica de la estación meteorológica:

# Busca la celda con información de la estación meteorológica
station=soup1.find('td', class_="b-metar-table__weather-station")
print("estación", station.text)  # Imprime nombre y ubicación de la estación

# Busca y extrae la temperatura
temp=soup1.find('span', class_="temp")
tem=temp.text
tem=tem + ' C'  # Añade la unidad de grados Celsius

# Busca y extrae la información del viento
wind=soup1.find('div', class_="b-metar-table__wind-text")
print("viento", wind.text)  # Imprime dirección y velocidad del viento

# Busca información sobre nubosidad y visibilidad
cloud_visi=soup1.find('div', class_="b-metar-table__additionally-container")
print("nubosidad", cloud_visi.text)  # Imprime información adicional sobre nubes

estación El Calvari wx 1 km NE22m alt.1 hour ago
viento Light winds from the W (14 km/h at W°).
nubosidad 


Vamos ahora a almacenar los datos.

In [21]:
# Importa el módulo csv para trabajar con archivos CSV
import csv

# Abre (o crea) un archivo CSV en modo append (añadir al final)
with open("Weather_Forecast.csv", "a", newline='') as file: # a es de append para añadir
    # Define los nombres de las columnas del CSV
    field_names = ['Weather Today days', 'Weather days', '10 Day Weather days', 'Weather_Station_Info', 'Temperature', 'Wind', 'Cloud_Visibility']
    # Crea un objeto writer que escribirá diccionarios con las columnas especificadas
    writer = csv.DictWriter(file, fieldnames=field_names)
    # Escribe la fila de encabezados con los nombres de las columnas
    writer.writeheader()

    # Escribe una fila de datos con toda la información meteorológica recopilada
    writer.writerow(
        {
            'Weather Today days': w_desc[0],  # Predicción para hoy
            'Weather days': w_desc[1],  # Predicción para los próximos días
            '10 Day Weather days': w_desc[2],  # Predicción a 10 días
            'Weather_Station_Info': station.text,  # Información de la estación
            'Temperature': tem,  # Temperatura actual
            'Wind': wind.text,  # Información del viento
            'Cloud_Visibility': cloud_visi.text  # Nubosidad y visibilidad
        }
    )